In this note, I will write how to preprocess the char level text with keras. 

First we preprocess the data to get training texts and labels.

In [37]:
import numpy as np
import pandas as pd

data_source = '../data/ag_news_csv/train.csv'

train_df = pd.read_csv(data_source, header=None)
train_df.head()

,0,1,2
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [2]:
# concatenate column 1 and column 2 as one text
train_df[1] = train_df[1] + train_df[2]
train_df = train_df.drop([2], axis=1)
train_df.head()

,0,1
0,3,Wall St. Bears Claw Back Into the Black (Reute...
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...
4,3,"Oil prices soar to all-time record, posing new..."


### train data to index

In [5]:
texts = train_df[1].values # texts contrain all setences
texts = [s.lower() for s in texts] # convert to lower case 
print(len(texts))
print(texts[:2])

120000
["wall st. bears claw back into the black (reuters)reuters - short-sellers, wall street's dwindling\\band of ultra-cynics, are seeing green again.", 'carlyle looks toward commercial aerospace (reuters)reuters - private investment firm carlyle group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.']


In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

When initializing the `Tokenizer`, there are only two parameter important.
- `char_level=True`: this can make later `tk.texts_to_sequences()` to process sentence in char level.
- `oov_token='UNK'`: this will add a `UNK` key in the vocabulary. We can call it by `tk.oov_token`.

In [19]:
# Initialization
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
# Fitting
tk.fit_on_texts(texts)

In [20]:
# List the vocabulary
tk.word_index

{' ': 1,
 '!': 54,
 '"': 48,
 '#': 34,
 '$': 50,
 '&': 42,
 "'": 39,
 '(': 36,
 ')': 37,
 '*': 56,
 ',': 25,
 '-': 26,
 '.': 22,
 '/': 44,
 '0': 29,
 '1': 35,
 '2': 38,
 '3': 28,
 '4': 46,
 '5': 45,
 '6': 47,
 '7': 49,
 '8': 51,
 '9': 31,
 ':': 43,
 ';': 27,
 '=': 52,
 '?': 53,
 'UNK': 57,
 '\\': 41,
 '_': 55,
 'a': 3,
 'b': 21,
 'c': 13,
 'd': 11,
 'e': 2,
 'f': 18,
 'g': 17,
 'h': 12,
 'i': 5,
 'j': 32,
 'k': 24,
 'l': 10,
 'm': 16,
 'n': 8,
 'o': 7,
 'p': 15,
 'q': 33,
 'r': 9,
 's': 6,
 't': 4,
 'u': 14,
 'v': 23,
 'w': 20,
 'x': 30,
 'y': 19,
 'z': 40}

But we already have a character list, so we do ont use `tk.word_index` as our vocabulary. We will replace it with `char_dict`.  

In [23]:
alphabet="abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1
char_dict

{'!': 41,
 '"': 45,
 '#': 51,
 '$': 52,
 '%': 53,
 '&': 55,
 "'": 44,
 '(': 64,
 ')': 65,
 '*': 56,
 '+': 59,
 ',': 38,
 '-': 60,
 '.': 40,
 '/': 46,
 '0': 27,
 '1': 28,
 '2': 29,
 '3': 30,
 '4': 31,
 '5': 32,
 '6': 33,
 '7': 34,
 '8': 35,
 '9': 36,
 ':': 43,
 ';': 39,
 '<': 62,
 '=': 61,
 '>': 63,
 '?': 42,
 '@': 50,
 '[': 66,
 '\\': 47,
 ']': 67,
 '^': 54,
 '_': 49,
 '`': 58,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 '{': 68,
 '|': 48,
 '}': 69,
 '~': 57}

In [24]:
# max index value
print(max(char_dict.values()))

69


In [25]:
# Use char_dict to replace the tk.word_index
tk.word_index = char_dict 
# Add 'UNK' to the vocabulary 
tk.word_index[tk.oov_token] = max(char_dict.values()) + 1 

In [26]:
tk.word_index

{'!': 41,
 '"': 45,
 '#': 51,
 '$': 52,
 '%': 53,
 '&': 55,
 "'": 44,
 '(': 64,
 ')': 65,
 '*': 56,
 '+': 59,
 ',': 38,
 '-': 60,
 '.': 40,
 '/': 46,
 '0': 27,
 '1': 28,
 '2': 29,
 '3': 30,
 '4': 31,
 '5': 32,
 '6': 33,
 '7': 34,
 '8': 35,
 '9': 36,
 ':': 43,
 ';': 39,
 '<': 62,
 '=': 61,
 '>': 63,
 '?': 42,
 '@': 50,
 'UNK': 70,
 '[': 66,
 '\\': 47,
 ']': 67,
 '^': 54,
 '_': 49,
 '`': 58,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 '{': 68,
 '|': 48,
 '}': 69,
 '~': 57}

After get the right vocabulary, we can convert all texts to index representation.

In [28]:
sequences = tk.texts_to_sequences(texts)
print(texts[0])
print(sequences[0])

wall st. bears claw back into the black (reuters)reuters - short-sellers, wall street's dwindling\band of ultra-cynics, are seeing green again.
[23, 1, 12, 12, 70, 19, 20, 40, 70, 2, 5, 1, 18, 19, 70, 3, 12, 1, 23, 70, 2, 1, 3, 11, 70, 9, 14, 20, 15, 70, 20, 8, 5, 70, 2, 12, 1, 3, 11, 70, 64, 18, 5, 21, 20, 5, 18, 19, 65, 18, 5, 21, 20, 5, 18, 19, 70, 60, 70, 19, 8, 15, 18, 20, 60, 19, 5, 12, 12, 5, 18, 19, 38, 70, 23, 1, 12, 12, 70, 19, 20, 18, 5, 5, 20, 44, 19, 70, 4, 23, 9, 14, 4, 12, 9, 14, 7, 47, 2, 1, 14, 4, 70, 15, 6, 70, 21, 12, 20, 18, 1, 60, 3, 25, 14, 9, 3, 19, 38, 70, 1, 18, 5, 70, 19, 5, 5, 9, 14, 7, 70, 7, 18, 5, 5, 14, 70, 1, 7, 1, 9, 14, 40]


In [30]:
# List top 5 sentence length
for i, s in enumerate(sequences):
    print(len(s))
    if i > 4:
        break

143
265
231
255
233
238


Because the sentence have different lengths, we have to make all sentence as a same length, so the CNN could handle the batch data. Here as the paper proposed, we set the sentence length as `1014`.

In [32]:
data = pad_sequences(sequences, maxlen=1014, padding='post')

In [36]:
print(sequences[0][:160])
print()
print(data[0][:160])

[23, 1, 12, 12, 70, 19, 20, 40, 70, 2, 5, 1, 18, 19, 70, 3, 12, 1, 23, 70, 2, 1, 3, 11, 70, 9, 14, 20, 15, 70, 20, 8, 5, 70, 2, 12, 1, 3, 11, 70, 64, 18, 5, 21, 20, 5, 18, 19, 65, 18, 5, 21, 20, 5, 18, 19, 70, 60, 70, 19, 8, 15, 18, 20, 60, 19, 5, 12, 12, 5, 18, 19, 38, 70, 23, 1, 12, 12, 70, 19, 20, 18, 5, 5, 20, 44, 19, 70, 4, 23, 9, 14, 4, 12, 9, 14, 7, 47, 2, 1, 14, 4, 70, 15, 6, 70, 21, 12, 20, 18, 1, 60, 3, 25, 14, 9, 3, 19, 38, 70, 1, 18, 5, 70, 19, 5, 5, 9, 14, 7, 70, 7, 18, 5, 5, 14, 70, 1, 7, 1, 9, 14, 40]

[23  1 12 12 70 19 20 40 70  2  5  1 18 19 70  3 12  1 23 70  2  1  3 11
 70  9 14 20 15 70 20  8  5 70  2 12  1  3 11 70 64 18  5 21 20  5 18 19
 65 18  5 21 20  5 18 19 70 60 70 19  8 15 18 20 60 19  5 12 12  5 18 19
 38 70 23  1 12 12 70 19 20 18  5  5 20 44 19 70  4 23  9 14  4 12  9 14
  7 47  2  1 14  4 70 15  6 70 21 12 20 18  1 60  3 25 14  9  3 19 38 70
  1 18  5 70 19  5  5  9 14  7 70  7 18  5  5 14 70  1  7  1  9 14 40  0
  0  0  0  0  0  0  0  0  0  0  0  0  0

In [38]:
data = np.array(data)
data.shape # All sentence length are 1014

(120000, 1014)

### get classes 

In [28]:
print(train_df[0].unique()) # how many unique classes
class_list = train_df[0].values
print(len(class_list)) # how many length
print(class_list[:10]) # print top 10 samples classes
# get class
class_list = [x-1 for x in class_list] # change index from [1, 2, 3, 4] to [0, 1, 2, 3] in order to use to_categorical()
print(set(class_list)) 
print(class_list[:10]) # print top 10 samples classes to comfirm the changes

[3 4 2 1]
120000
[3 3 3 3 3 3 3 3 3 3]
{0, 1, 2, 3}
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [29]:
from keras.utils import to_categorical
classes = to_categorical(class_list)
print(classes[:10])

[[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]]


In [40]:
ls ../data/ag_news_csv/

classes.txt* readme.txt*  test.csv*    train.csv*


In [41]:
# write all code in one cell 

#========================Load data=========================
import numpy as np
import pandas as pd

train_data_source = '../data/ag_news_csv/train.csv'
test_data_source = '../data/ag_news_csv/test.csv'

train_df = pd.read_csv(train_data_source, header=None)
test_df = pd.read_csv(test_data_source, header=None)

# concatenate column 1 and column 2 as one text
for df in [train_df, test_df]:
    df[1] = df[1] + df[2]
    df = df.drop([2], axis=1)
    
# convert string to lower case 
train_texts = train_df[1].values 
train_texts = [s.lower() for s in train_texts] 

test_texts = test_df[1].values 
test_texts = [s.lower() for s in test_texts] 

#=======================Convert string to index================
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Tokenizer
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(train_texts)
# If we already have a character list, then replace the tk.word_index
# If not, just skip below part

#-----------------------Skip part start--------------------------
# construct a new vocabulary 
alphabet="abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1
    
# Use char_dict to replace the tk.word_index
tk.word_index = char_dict 
# Add 'UNK' to the vocabulary 
tk.word_index[tk.oov_token] = max(char_dict.values()) + 1
#-----------------------Skip part end----------------------------

# Convert string to index 
train_sequences = tk.texts_to_sequences(train_texts)
test_texts = tk.texts_to_sequences(test_texts)

# Padding
train_data = pad_sequences(train_sequences, maxlen=1014, padding='post')
test_data = pad_sequences(test_texts, maxlen=1014, padding='post')

# Convert to numpy array
train_data = np.array(train_data)
test_data = np.array(test_data)

#=======================Get classes================
train_classes = train_df[0].values
train_class_list = [x-1 for x in train_classes]

test_classes = test_df[0].values
test_class_list = [x-1 for x in test_classes]

from keras.utils import to_categorical
train_classes = to_categorical(train_class_list)
test_classes = to_categorical(test_class_list)